In [2]:
# HW 04 -- Pandas Challenge -- Heroes of Pymoli
import pandas as pd
import numpy as np
import os

In [3]:
# find path and read the csv
pymoli_path = "./purchase_data.csv"

In [111]:
pymoli_df = pd.read_csv(pymoli_path)
pymoli_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [5]:
#count players ID to determine the total players
total_player = len(pymoli_df['SN'].value_counts())
total_player

576

In [6]:
#count unique items
unique_items = pymoli_df['Item Name'].nunique()
unique_items

179

In [7]:
#count average price
average_price = round(pymoli_df['Price'].mean(),2)
average_price

3.05

In [8]:
# count number of purchases
n_purchase = len(pymoli_df['Purchase ID'].value_counts())
n_purchase

780

In [9]:
# count the total revenue
total_revenue = round(pymoli_df['Price'].sum(),2)
total_revenue

2379.77

In [66]:
# make a summary data frame to show all above results: 

total_df = pd.DataFrame({"Total Players":total_player,
                         "Number of Unique Items":unique_items,
                         "Average Price":average_price,
                         "Number of Purchases":n_purchase,
                         "Total Revenue":total_revenue}, index=[0])
total_df = total_df.style.format({'Average Price':'${0:,.2f}',
                      'Total Revenue':'${0:,.2f}'})
total_df

,Total Players,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,576,179,$3.05,780,"$2,379.77"


In [137]:
#Gender Demographics
# Group by gender, then count players (by unique ID) and calculate the percentage: 

gender_df = pymoli_df.groupby("Gender")['SN'].agg([('players count',lambda x: x.nunique())])


# Format the table (use mapping)
gender_df["Percentage"] = gender_df["players count"] / gender_df["players count"].sum()
gender_df["Percentage"] = gender_df["Percentage"].map("{:,.2%}".format)
gender_df["players count"] = gender_df["players count"].map("{:,.0f}".format)


gender_df


,players count,Percentage
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [24]:
# Now we know there are 81 Female palyers, 484 Male players, and 11 Other players

In [138]:
#Purchasing Analysis (Gender)
# group by gender, and use agg to find out purchase count, average purchase price, and total purchase value:
gender_p_df = pymoli_df.groupby("Gender")['Price'].agg([('count',np.count_nonzero),('mean',np.mean),
                                                       ('sum',np.sum)])

# as we already knew the number of players by gender, we can calculate the average purchase per person
gender_p_df["players count"] = [81,484,11]
gender_p_df["Avg per person"] = gender_p_df["sum"] / gender_p_df["players count"]

# format the data
gender_p_df = gender_p_df.style.format({'mean':'${0:,.2f}',
                      'sum':'${0:,.2f}','Avg per person':'${0:,.2f}', 'count':'{0:,.0f}' })

gender_p_df

,count,mean,sum,players count,Avg per person
Gender,,,,,
Female,113,$3.20,$361.94,81,$4.47
Male,652,$3.02,"$1,967.64",484,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,11,$4.56


In [67]:
# Age Demographics  ** Data Frame = df
# Please note that in this section I put players count & percentage df with the purchasing analysis df together.

# make a bin range that holds all ages, but need to know min and max age first: 

pymoli_df["Age"].min(),pymoli_df["Age"].max()

(7, 45)

In [85]:
bins_range = list(range(pymoli_df["Age"].min()-2,pymoli_df["Age"].max()+1,5))
bins_range

[5, 10, 15, 20, 25, 30, 35, 40, 45]

In [130]:
# now name each range and put data into each range
bin_names = ["<=10","11-15","16-20","21-25","26-30","31-35","36-40","41-45"]
pymoli_df["Age range"] = pd.cut(pymoli_df["Age"],bins_range,labels = bin_names,include_lowest=True)


# make a Age data frame that grouped raw data by age, and get basic purchase calculations, count, mean, and sum: 
age_df = pymoli_df.groupby("Age range")['Price'].agg([('purchase count',np.count_nonzero),('Avg purchase',np.mean),
                                                     ('sum',np.sum)])

#count players of each age range and calculate percentage 
age_count_df = pymoli_df.groupby("Age range")['SN'].agg([('players count',lambda y: y.nunique())])
age_count_df["Players Percentage"] = age_count_df["players count"] / age_count_df["players count"].sum()

age_df["players count"] = age_count_df["players count"]
age_df["players percentage"] = age_count_df["Players Percentage"]

# calculate the average purchase per person by age group
age_df["Avg purchase per person"] = age_df["sum"] / age_df["players count"]


# format the Age df
age_df = age_df.style.format({'Avg purchase':'${0:,.2f}',
                      'sum':'${0:,.2f}','Avg purchase per person':'${0:,.2f}', "players percentage":'{:,.2%}' ,
                             'purchase count':'{0:,.0f}'})

age_df

,purchase count,Avg purchase,sum,players count,players percentage,Avg purchase per person
Age range,,,,,,
<=10,32,$3.40,$108.96,24,4.17%,$4.54
11-15,54,$2.90,$156.60,41,7.12%,$3.82
16-20,200,$3.11,$621.56,150,26.04%,$4.14
21-25,325,$3.02,$981.64,232,40.28%,$4.23
26-30,77,$2.88,$221.42,59,10.24%,$3.75
31-35,52,$2.99,$155.71,37,6.42%,$4.21
36-40,33,$3.40,$112.35,26,4.51%,$4.32
41-45,7,$3.08,$21.53,7,1.22%,$3.08


In [125]:
# Top Spenders
# make a df for top 5 spenders (grouped by SN)
top_df = pymoli_df.groupby("SN")['Price'].agg([('purchase count',np.count_nonzero),('Avg purchase',np.mean),
                                                     ('sum',np.sum)])

# sort the data from high to low (total purchased amount, aka sum)
top_df = top_df.sort_values(["sum"],ascending=False)

# Format the table (use mapping):
top_df["Avg purchase"] = top_df["Avg purchase"].map("${:.2f}".format)
top_df["sum"] = top_df["sum"].map("${:.2f}".format)
top_df["purchase count"] = top_df["purchase count"].map("{:.0f}".format)

#show top 5
top_df[:5]


,purchase count,Avg purchase,sum
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [141]:
# Most Popular Items
# Group by Item ID and Item Name: 
popular_df = pymoli_df.groupby(["Item ID","Item Name"])['Price'].agg([('purchase count',np.count_nonzero),('Avg purchase',np.mean),
                                                     ('sum',np.sum)])
popular_df = popular_df.sort_values(["purchase count"],ascending=False)

# Format the table (use mapping):
popular_df["Avg purchase"] = popular_df["Avg purchase"].map("${:.2f}".format)
popular_df["sum"] = popular_df["sum"].map("${:.2f}".format)
popular_df["purchase count"] = popular_df["purchase count"].map("{:.0f}".format)


popular_df[:5]

,,purchase count,Avg purchase,sum
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [142]:
# Most Profitable Items
profit_df = pymoli_df.groupby(["Item ID","Item Name"])['Price'].agg([('purchase count',np.count_nonzero),('Avg purchase',np.mean),
                                                     ('sum',np.sum)])
profit_df = profit_df.sort_values(["sum"],ascending=False)

# Format the table (use mapping):
profit_df["Avg purchase"] =profit_df["Avg purchase"].map("${:.2f}".format)
profit_df["sum"] = profit_df["sum"].map("${:.2f}".format)
profit_df["purchase count"] = profit_df["purchase count"].map("{:.0f}".format)


profit_df[:5]


,,purchase count,Avg purchase,sum
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
